## Importações e Configuração Inicial
 - AJUSTAR PARA BUSCAR INFORMAÇÕES DE PONTUAÇÕES MENSAIS

In [7]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
from pathlib import Path
from bs4 import BeautifulSoup
import json, re
import time

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [8]:
# 👉 ajuste aqui se precisar
HTML_PATH = Path("pagina_liga_classica.html")  

# carrega html
soup = BeautifulSoup(HTML_PATH.read_text(encoding="utf-8", errors="ignore"), "html.parser")

# coleta nomes (principal + fallback)
nomes = [el.get_text(strip=True) for el in soup.select("p.nome-time")]
if not nomes:
    nomes = [(a.get("title") or a.get_text(strip=True)).strip()
             for a in soup.select("a.link-perfil-time") if (a.get("title") or a.get_text(strip=True))]

# dedup + ordena alfabético
visto, times = set(), []
for n in nomes:
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML
OUT_JS = HTML_PATH.with_name("participantesLiga_Classica.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n", encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")


59 times encontrados
['A Lenda Super Vasco F.c', 'A Lenda Super Vascão f.c', 'Analove10 ITAQUI GRANDE!!', 'BordonFC', 'BORGES CLIMA FUT F.C', 'BORGES ITAQUI F.C.', 'C.A. Charru@', 'CALOMBO ITAQUI RS', 'cartola scheuer', 'clarinvalau fc', 'CosmoCity ZO', 'DM Studio', 'Dom Camillo68', 'emer jr fc', 'FBC Colorado', 'Fedato Futebol Clube', 'FIGUEIRA DA ILHA', 'Gig@ntte', 'Gremiomaniasm', 'HS SPORTS F.C', 'JV5 Tricolor Gaúcho', 'KING LEONN', 'KP JUV.', 'Laranjja Mecannica', 'lsauer fc', 'Luis lemes  inter', 'MAFRA MARTINS FC', 'MauHumor F.C.', 'mercearia Estrela', 'Noah A 10', 'O clube do povo Itaqui/Rss', 'OlhaEleAiF.C!', 'Paulo Virgili FC', 'Perronee F.C', 'Pity10', 'pra sempre imortal fc', 'PraiaBravaFC', 'Profit Soccer', 'pura bucha /botafogo', 'PUXE FC', 'Real SCI', 'RHANKA DENTY FC25', 'Rolo Compressor  ZN', 'RS Expressões da Arte', 'S.E.R. GRILLO', 'seralex', 'SERGRILLO', 'Sport Clube PAIM', 'Super Vasco f.c', 'SUPER VASCÃO F.C', 'Tabajara de Inhaua FC2', 'TATITTA FC', 'Tatols Beants

In [9]:
JS_PATH = Path("participantesLiga_Classica.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times[:10]

59 times


['A Lenda Super Vasco F.c',
 'A Lenda Super Vascão f.c',
 'Analove10 ITAQUI GRANDE!!',
 'BordonFC',
 'BORGES CLIMA FUT F.C',
 'BORGES ITAQUI F.C.',
 'C.A. Charru@',
 'CALOMBO ITAQUI RS',
 'cartola scheuer',
 'clarinvalau fc']

In [10]:

# nomes_times = ["Real SCI", "Gig@ntte", "PraiaBravaFC", "OlhaEleAiF.C!", "Gremiomaniasm", "Sport Clube PAIM", "PUXE FC", "RS Expressões da Arte", "ZIVI FC", "O clube do povo Itaqui/Rss",
#                "F.C. Rei Das Copas", "Rolo Compressor ZN", "Taura da Fronteira FCIII", "Eleis-Itaqui", "KING LEONN", "Laranjja Mecannica", "Fedato Futebol Clube", "Perronee F.C", "Pity10",
#                "pra sempre imortal fc", "RHANKA DENTY FC25", "TEAM LOPES 99", "pura bucha /botafogo", "cartola scheuer", "Analove10 ITAQUI GRANDE!!", "DM Studio",
#                "lsauer fc", "VASCO MARTINS FC", "KP JUV.", "BORGES ITAQUI F.C.", "Profit Soccer", "Tabajara de Inhaua FC2", "TIGRE LEON", "S.E.R. GRILLO", "seralex", "E.C. Bororé",
#                "Texas Club 2025", "MAFRA MARTINS FC", "BordonFC", "Tatols Beants F.C", "FIGUEIRA DA ILHA", "MauHumor F.C.", "A Lenda Super Vascão f.c", "TATITTA FC", "HS SPORTS F.C",
#                "FBC Colorado", "Dom Camillo68", "mercearia Estrela", "CosmoCity ZO", "clarinvalau fc", "Grêmio imortal 37", "SERGRILLO", "Super Vasco f.c", "A Lenda Super Vasco F.c", 
#                "Paulo Virgili FC", "CALOMBO ITAQUI RS", "Luis lemes inter", "emer jr fc", "Cril Futebol Club", "Santo Ângelo United", "Remo Santo Ângelo"]


# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função para buscar o ID de um time pelo nome usando correspondência flexível
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]
        nome_proximo = get_close_matches(nome_time, nomes_api, n=1, cutoff=0.6)
        if nome_proximo:
            for time in times:
                if time.nome == nome_proximo[0]:
                    return time.id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar os IDs dos times pelo nome
for nome in nomes_times:
    time_id = buscar_id_time(nome)
    if time_id:
        ids_times[nome] = time_id
    else:
        print(f"Não foi possível encontrar o ID para o time {nome}")

print(nomes_times)

['A Lenda Super Vasco F.c', 'A Lenda Super Vascão f.c', 'Analove10 ITAQUI GRANDE!!', 'BordonFC', 'BORGES CLIMA FUT F.C', 'BORGES ITAQUI F.C.', 'C.A. Charru@', 'CALOMBO ITAQUI RS', 'cartola scheuer', 'clarinvalau fc', 'CosmoCity ZO', 'DM Studio', 'Dom Camillo68', 'emer jr fc', 'FBC Colorado', 'Fedato Futebol Clube', 'FIGUEIRA DA ILHA', 'Gig@ntte', 'Gremiomaniasm', 'HS SPORTS F.C', 'JV5 Tricolor Gaúcho', 'KING LEONN', 'KP JUV.', 'Laranjja Mecannica', 'lsauer fc', 'Luis lemes  inter', 'MAFRA MARTINS FC', 'MauHumor F.C.', 'mercearia Estrela', 'Noah A 10', 'O clube do povo Itaqui/Rss', 'OlhaEleAiF.C!', 'Paulo Virgili FC', 'Perronee F.C', 'Pity10', 'pra sempre imortal fc', 'PraiaBravaFC', 'Profit Soccer', 'pura bucha /botafogo', 'PUXE FC', 'Real SCI', 'RHANKA DENTY FC25', 'Rolo Compressor  ZN', 'RS Expressões da Arte', 'S.E.R. GRILLO', 'seralex', 'SERGRILLO', 'Sport Clube PAIM', 'Super Vasco f.c', 'SUPER VASCÃO F.C', 'Tabajara de Inhaua FC2', 'TATITTA FC', 'Tatols Beants F.C', 'Taura da Fron

In [11]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_classica.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls)


✅ Arquivo salvo com sucesso: links_times_cartola_liga_classica.xlsx


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,A Lenda Super Vascão f.c,49355335,https://cartola.globo.com/#!/time/49355335
2,Analove10 ITAQUI GRANDE!!,44574236,https://cartola.globo.com/#!/time/44574236
3,BordonFC,2602301,https://cartola.globo.com/#!/time/2602301
4,BORGES CLIMA FUT F.C,49852616,https://cartola.globo.com/#!/time/49852616
5,BORGES ITAQUI F.C.,3914981,https://cartola.globo.com/#!/time/3914981
6,C.A. Charru@,25748736,https://cartola.globo.com/#!/time/25748736
7,CALOMBO ITAQUI RS,8707471,https://cartola.globo.com/#!/time/8707471
8,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
9,clarinvalau fc,8319052,https://cartola.globo.com/#!/time/8319052


In [12]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual

print("Rodada Atual:", rodada_atual)

Rodada Atual: 20


## Obter Pontuações por Rodada

In [13]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual

# Função para verificar se o campeonato já começou (há pontuação na rodada 1)
def campeonato_comecou(ids_times):
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

# Verifica se o campeonato começou
if not campeonato_comecou(ids_times):
    print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")

    total_rodadas = 38
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
    df_pontuacoes[:] = 0  # ou None
else:
    def obter_pontuacao_por_rodada(time_id, rodada_atual):
        pontuacoes = {}
        for rodada in range(1, rodada_atual):
            try:
                time_rodada = api.time(time_id=time_id, rodada=rodada)
                pontuacoes[rodada] = time_rodada.ultima_pontuacao
            except cartolafc.errors.CartolaFCError as e:
                print(f"Erro ao acessar a pontuação da rodada {rodada} para o time {time_id}: {e}")
                pontuacoes[rodada] = None
        return pontuacoes

    df_pontuacoes = pd.DataFrame()

    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(time_id, rodada_atual)
        df_pontuacoes[nome] = pd.Series(pontuacoes)

    df_pontuacoes = df_pontuacoes.transpose()
    df_pontuacoes.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]

    lider_rodada = df_pontuacoes.idxmax()
    df_pontuacoes.loc['Lider_Rodada'] = lider_rodada

# ===============================
# 📊 Segmentação por Turnos e Meses
# ===============================

rodadas_por_mes = {
    # "Março": [1],
    # "Abril": [2, 3, 4, 5, 6],
    # "Maio": [7, 8, 9, 10, 11],
    # "Junho": [12],
    # "Julho": [13, 14, 15, 16, 17],
    "Agosto": [18, 19, 20, 21, 22],
    "Setembro": [23, 24, 25],
    "Outubro": [26, 27, 28],
    "Novembro": [29, 30, 31, 32],
    "Dezembro": [33, 34, 35, 36, 37, 38],
}

# rodadas_primeiro_turno = list(range(1, 20))
rodadas_segundo_turno = list(range(20, 39))

# Função para filtrar e somar pontuações
def somar_pontuacoes(df, rodadas):
    colunas = [f'Rodada {i}' for i in rodadas if f'Rodada {i}' in df.columns]
    return df[colunas].drop(index='Lider_Rodada', errors='ignore').sum(axis=1).sort_values(ascending=False)

# DataFrames por turno
# df_turno_1 = df_pontuacoes[[f'Rodada {i}' for i in rodadas_primeiro_turno if f'Rodada {i}' in df_pontuacoes.columns]]
df_turno_2 = df_pontuacoes[[f'Rodada {i}' for i in rodadas_segundo_turno if f'Rodada {i}' in df_pontuacoes.columns]]

# Soma por turno
# df_soma_turno_1 = somar_pontuacoes(df_pontuacoes, rodadas_primeiro_turno)
df_soma_turno_2 = somar_pontuacoes(df_pontuacoes, rodadas_segundo_turno)

# Soma por mês
df_soma_mensal = {}
for mes, rodadas in rodadas_por_mes.items():
    df_soma_mensal[mes] = somar_pontuacoes(df_pontuacoes, rodadas)

# ===============================
# 📁 Salvar tudo em Excel
# ===============================
with pd.ExcelWriter("Pontuacoes_Liga_Classica_Completa.xlsx") as writer:
    df_pontuacoes.to_excel(writer, sheet_name="Geral")
    # df_turno_1.to_excel(writer, sheet_name="Turno 1")
    df_turno_2.to_excel(writer, sheet_name="Turno 2")
    # pd.DataFrame(df_soma_turno_1).to_excel(writer, sheet_name="Classif Turno 1")
    pd.DataFrame(df_soma_turno_2).to_excel(writer, sheet_name="Classif Turno 2")
    for mes, serie in df_soma_mensal.items():
        pd.DataFrame(serie).to_excel(writer, sheet_name=f"Mês - {mes}")

print("✅ Arquivo Excel salvo com todas as segmentações.")

# ===============================
# 💾 Salvar em JS
# ===============================
df_dict_geral = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').fillna(0).to_dict(orient="index")

df_js = {
    "geral": df_dict_geral,
    "turnos": {
        # "turno_1": df_soma_turno_1.to_dict(),
        "turno_2": df_soma_turno_2.to_dict(),
    },
    "meses": {mes: serie.to_dict() for mes, serie in df_soma_mensal.items()}
}

js_content = f"const classificacaoLigaClassica = {json.dumps(df_js, indent=2, ensure_ascii=False)};"

with open("Pontuacoes_Liga_Classica.js", "w", encoding="utf-8") as f:
    f.write(js_content)

print("✅ Arquivo JS salvo com estrutura geral, turnos e meses.")

# Exibir a pontuação geral transposta
display(df_pontuacoes.T)


✅ Arquivo Excel salvo com todas as segmentações.
✅ Arquivo JS salvo com estrutura geral, turnos e meses.


C:\Users\ferna\AppData\Local\Temp\ipykernel_36768\3521999803.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dict_geral = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').fillna(0).to_dict(orient="index")


,A Lenda Super Vasco F.c,A Lenda Super Vascão f.c,Analove10 ITAQUI GRANDE!!,BordonFC,BORGES CLIMA FUT F.C,BORGES ITAQUI F.C.,C.A. Charru@,CALOMBO ITAQUI RS,cartola scheuer,clarinvalau fc,CosmoCity ZO,DM Studio,Dom Camillo68,emer jr fc,FBC Colorado,Fedato Futebol Clube,FIGUEIRA DA ILHA,Gig@ntte,Gremiomaniasm,HS SPORTS F.C,JV5 Tricolor Gaúcho,KING LEONN,KP JUV.,Laranjja Mecannica,lsauer fc,...,pra sempre imortal fc,PraiaBravaFC,Profit Soccer,pura bucha /botafogo,PUXE FC,Real SCI,RHANKA DENTY FC25,Rolo Compressor ZN,RS Expressões da Arte,S.E.R. GRILLO,seralex,SERGRILLO,Sport Clube PAIM,Super Vasco f.c,SUPER VASCÃO F.C,Tabajara de Inhaua FC2,TATITTA FC,Tatols Beants F.C,Taura da Fronteira FCIII,TEAM LOPES 99,teves_futsal20 f.c,Texas Club 2025,TIGRE LEON,VASCO MARTINS FC,Lider_Rodada
Rodada 1,59.00,66.15,72.59,66.69,64.09,71.09,78.80,56.38,73.09,63.08,63.49,72.55,63.65,50.03,64.00,77.58,66.50,100.34,95.23,64.79,64.55,79.30,72.24,78.80,72.38,...,74.69,96.39,70.19,73.78,84.69,104.09,74.69,81.95,83.99,68.48,68.40,61.15,93.85,60.55,59.30,69.45,65.40,66.60,80.25,74.09,64.48,67.69,69.40,72.29,Real SCI
Rodada 2,88.29,76.48,76.19,74.39,93.19,71.39,68.77,80.62,86.39,60.87,91.42,81.69,67.69,60.88,72.69,77.49,103.62,86.39,59.22,94.48,44.92,93.04,103.19,96.79,79.29,...,68.19,96.19,87.89,73.84,95.29,75.69,86.49,74.49,92.82,81.12,72.89,104.17,67.39,96.99,72.58,87.99,59.68,73.32,80.59,73.19,82.28,85.99,83.04,56.82,SERGRILLO
Rodada 3,63.70,47.22,67.40,75.42,62.90,56.40,70.90,47.35,75.10,74.02,63.12,82.67,71.20,56.00,81.52,75.42,76.90,86.80,95.55,65.12,65.10,84.70,92.35,80.10,82.17,...,63.30,87.05,77.92,77.12,86.95,87.02,70.10,56.00,68.52,53.84,38.22,40.77,85.52,71.40,60.70,72.95,61.70,99.55,58.62,68.42,96.80,76.92,83.60,91.99,Tatols Beants F.C
Rodada 4,86.72,97.33,119.31,93.18,85.16,99.68,81.03,102.56,77.36,80.26,94.87,106.62,136.21,72.37,78.62,90.16,76.51,79.58,85.67,108.16,80.62,84.48,75.63,63.93,104.93,...,79.03,81.06,80.06,88.32,85.01,99.66,98.48,76.03,77.28,60.93,77.06,51.63,91.70,90.63,91.63,94.56,90.15,94.87,96.53,109.06,115.77,79.86,61.83,62.22,Dom Camillo68
Rodada 5,81.72,88.58,86.62,82.73,68.42,67.62,64.21,56.88,80.92,60.15,93.47,73.53,103.57,54.51,69.38,81.02,70.41,73.01,86.03,83.48,77.33,72.61,84.61,74.52,66.22,...,74.37,94.26,82.18,76.42,73.92,82.42,86.22,53.52,57.92,72.33,69.92,84.68,86.48,85.32,72.78,91.02,58.43,83.03,84.98,87.27,67.53,71.32,62.41,72.02,Dom Camillo68
Rodada 6,93.36,106.93,104.20,131.61,106.15,109.65,86.66,61.88,98.30,77.48,111.43,86.06,78.56,65.54,92.05,92.06,110.36,78.86,78.92,82.65,100.05,99.56,98.06,49.26,108.56,...,102.86,80.06,110.32,77.50,88.76,103.26,98.16,68.45,84.05,95.32,66.76,69.61,88.06,100.86,98.23,106.15,73.26,86.06,98.80,106.06,84.40,119.85,85.16,48.84,BordonFC
Rodada 7,94.09,85.49,106.49,123.84,111.86,92.99,93.99,65.14,95.79,95.89,93.31,133.81,119.89,123.81,94.59,104.31,93.54,90.36,93.74,98.19,120.84,87.66,85.69,79.66,87.71,...,116.56,75.89,96.36,98.79,96.11,94.34,96.69,85.84,85.81,86.09,99.29,92.36,97.59,96.39,87.19,113.89,86.24,105.71,83.79,127.76,116.29,98.71,100.16,114.66,DM Studio
Rodada 8,131.06,128.44,135.19,102.44,150.95,80.89,132.25,73.34,99.45,45.04,115.29,92.98,93.04,86.43,122.84,98.49,112.35,98.15,73.51,92.49,114.14,98.45,110.15,125.15,118.45,...,92.29,66.00,120.74,76.34,103.93,108.45,106.65,94.80,89.40,46.90,136.94,81.15,100.69,121.94,110.69,135.49,73.29,97.39,112.24,139.40,98.45,109.15,117.55,91.35,BORGES CLIMA FUT F.C
Rodada 9,84.40,66.77,74.32,59.62,95.92,88.40,67.42,52.08,73.27,58.90,79.67,60.45,68.35,65.65,58.92,62.97,76.02,65.42,64.00,91.57,76.32,60.42,62.12,54.72,69.15,...,89.70,24.63,76.47,81.70,77.90,60.57,74.25,71.57,65.63,71.22,50.32,51.67,79.07,73.75,85.40,56.87,66.73,75.55,67.70,78.32,97.00,83.90,69.48,114.33,VASCO MARTINS FC
Rodada 10,69.67,66.07,79.87,54.77,69.27,65.97,57.67,68.02,73.57,49.27,72.27,69.57,64.50,71.50,68.97,66.07,65.47,53.97,53.37,66.47,71.97,40.77,57.62,44.37,57.17,...,64.67,54.12,60.02,68.67,61.47,50.37,73.57,72.52,59.57,20.85,63.92,42.11,88.17,76.97,45.57

In [14]:
# Exibir a pontuação
display(df_pontuacoes)

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
A Lenda Super Vasco F.c,59.00,88.29,63.70,86.72,81.72,93.36,94.09,131.06,84.40,69.67,110.27,79.36,111.91,75.80,88.30,78.00,51.94,89.80,66.05
A Lenda Super Vascão f.c,66.15,76.48,47.22,97.33,88.58,106.93,85.49,128.44,66.77,66.07,110.41,72.41,118.27,80.23,100.50,70.06,49.71,79.60,65.10
Analove10 ITAQUI GRANDE!!,72.59,76.19,67.40,119.31,86.62,104.20,106.49,135.19,74.32,79.87,105.54,69.91,118.74,75.40,104.20,75.06,58.51,78.87,69.10
BordonFC,66.69,74.39,75.42,93.18,82.73,131.61,123.84,102.44,59.62,54.77,134.67,64.84,119.24,65.10,86.60,77.47,48.74,81.87,78.95
BORGES CLIMA FUT F.C,64.09,93.19,62.90,85.16,68.42,106.15,111.86,150.95,95.92,69.27,114.54,62.26,96.51,64.80,91.60,76.66,36.89,80.60,54.63
BORGES ITAQUI F.C.,71.09,71.39,56.40,99.68,67.62,109.65,92.99,80.89,88.40,65.97,91.91,66.60,105.91,66.50,89.40,63.78,55.71,70.00,55.00
C.A. Charru@,78.80,68.77,70.90,81.03,64.21,86.66,93.99,132.25,67.42,57.67,63.85,56.61,106.27,47.60,82.41,62.60,69.74,82.20,65.85
CALOMBO ITAQUI RS,56.38,80.62,47.35,102.56,56.88,61.88,65.14,73.34,52.08,68.02,81.56,29.85,58.40,65.83,90.12,26.50,94.05,79.38,30.12
cartola scheuer,73.09,86.39,75.10,77.36,80.92,98.30,95.79,99.45,73.27,73.57,108.04,69.26,132.21,74.20,101.80,70.17,52.84,94.87,69.60
clarinvalau fc,63.08,60.87,74.02,80.26,60.15,77.48,95.89,45.04,58.90,49.27,66.66,35.88,92.41,18.80,90.89,58.25,63.20,37.86,63.85


In [15]:
for mes, dados in df_js["meses"].items():
    # Verifica se existem pontuações válidas (> 0)
    pontuacoes_validas = [p for p in dados.values() if p not in (None, 0)]
    
    if not pontuacoes_validas:
        continue  # pula este mês se todos os valores forem 0 ou None

    print(f"\n🏆 Top 5 de {mes.capitalize()}:")

    top_5 = sorted(dados.items(), key=lambda x: x[1], reverse=True)[:5]

    for i, (nome, pontos) in enumerate(top_5, 1):
        print(f"{i}º - {nome}: {pontos:.2f} pontos")


🏆 Top 5 de Agosto:
1º - TEAM LOPES 99: 192.55 pontos
2º - Laranjja Mecannica: 187.63 pontos
3º - FBC Colorado: 183.50 pontos
4º - CosmoCity ZO: 182.30 pontos
5º - HS SPORTS F.C: 181.00 pontos


In [16]:
from collections import defaultdict

# Etapa 1 – calculando pontuações e armazenando posições
pontuacao_por_posicao = {1: 50, 2: 40, 3: 30, 4: 20, 5: 10}
pontuacao_times = defaultdict(int)
posicoes_por_time = defaultdict(list)

for mes, dados in df_js["meses"].items():
    pontuacoes_validas = [p for p in dados.values() if p not in (None, 0)]
    if not pontuacoes_validas:
        continue

    top_5 = sorted(dados.items(), key=lambda x: x[1], reverse=True)[:5]

    for posicao, (nome, _) in enumerate(top_5, start=1):
        pontos = pontuacao_por_posicao.get(posicao, 0)
        pontuacao_times[nome] += pontos
        posicoes_por_time[nome].append((mes, posicao))

# Etapa 2 – unificar em um único dicionário
ranking_unificado = {}

for nome in set(pontuacao_times) | set(posicoes_por_time):
    ranking_unificado[nome] = {
        "pontos": pontuacao_times.get(nome, 0),
        "aparicoes": len(posicoes_por_time.get(nome, [])),
        "posicoes": posicoes_por_time.get(nome, [])
    }

# Etapa 3 – ordenar por pontuação
ranking_ordenado = sorted(ranking_unificado.items(), key=lambda x: x[1]["pontos"], reverse=True)

# Exibir
print("🏆 Ranking geral com posições por mês:")
for i, (nome, info) in enumerate(ranking_ordenado, 1):
    print(f"\n{i}º - {nome}: {info['pontos']} pontos em {info['aparicoes']} meses")
    for mes, pos in info['posicoes']:
        print(f"  - {mes}: {pos}º lugar")


🏆 Ranking geral com posições por mês:

1º - TEAM LOPES 99: 50 pontos em 1 meses
  - Agosto: 1º lugar

2º - Laranjja Mecannica: 40 pontos em 1 meses
  - Agosto: 2º lugar

3º - FBC Colorado: 30 pontos em 1 meses
  - Agosto: 3º lugar

4º - CosmoCity ZO: 20 pontos em 1 meses
  - Agosto: 4º lugar

5º - HS SPORTS F.C: 10 pontos em 1 meses
  - Agosto: 5º lugar


In [17]:
# Montar os dados no formato de lista
ranking_para_js = []

for nome, info in ranking_unificado.items():
    ranking_para_js.append({
        "time": nome,
        "pontos": info["pontos"],
        "aparicoes": info["aparicoes"],
        "posicoes": [
            {"mes": mes, "posicao": posicao} for mes, posicao in info["posicoes"]
        ]
    })

# Salvar como .js com a constante JavaScript
with open("ranking_top5_mensal.js", "w", encoding="utf-8") as f:
    f.write("const rankingTop5Mensal = ")
    json.dump(ranking_para_js, f, indent=2, ensure_ascii=False)
    f.write(";")

print("✅ Arquivo ranking_top5_mensal.js gerado com sucesso!")


✅ Arquivo ranking_top5_mensal.js gerado com sucesso!


In [18]:
import json
import os

# Montar os dados no formato de lista
ranking_para_js = []

for nome, info in ranking_unificado.items():
    ranking_para_js.append({
        "time": nome,
        "pontos": info["pontos"],
        "aparicoes": info["aparicoes"],
        "posicoes": [
            {"mes": mes, "posicao": posicao} for mes, posicao in info["posicoes"]
        ]
    })

# Nome do arquivo
nome_arquivo = "ranking_top5_mensal.js"

# Caminhos:
# 1. Atual (pasta onde o script está sendo executado)
# 2. Volta 2 níveis (.. -> datasets_liga_classica, .. -> liga_classica) e entra em campeoes/datasets_campeoes
caminhos_destino = [
    nome_arquivo,  # atual
    os.path.join("..", "..", "campeoes", "datasets_campeoes", nome_arquivo)  # outro destino
]

# Criar a pasta destino se não existir
os.makedirs(os.path.join("..", "..", "campeoes", "datasets_campeoes"), exist_ok=True)

# Salvar o arquivo nos dois locais
for caminho in caminhos_destino:
    with open(caminho, "w", encoding="utf-8") as f:
        f.write("const rankingTop5Mensal = ")
        json.dump(ranking_para_js, f, indent=2, ensure_ascii=False)
        f.write(";")
    print(f"✅ Arquivo salvo com sucesso: {caminho}")


✅ Arquivo salvo com sucesso: ranking_top5_mensal.js
✅ Arquivo salvo com sucesso: ..\..\campeoes\datasets_campeoes\ranking_top5_mensal.js


In [19]:
# Preparar os dados para o DataFrame
dados_df = []

for nome, info in ranking_unificado.items():
    meses_formatados = ', '.join([f"{mes} ({pos}º)" for mes, pos in info["posicoes"]])
    dados_df.append({
        "Time": nome,
        "Pontos": info["pontos"],
        "Aparições no Top 5": info["aparicoes"],
        "Meses e Posições": meses_formatados
    })

# Criar o DataFrame
df_ranking = pd.DataFrame(dados_df)

# Ordenar pela pontuação
df_ranking = df_ranking.sort_values(by="Pontos", ascending=False).reset_index(drop=True)

# Exibir o DataFrame
display(df_ranking)


,Time,Pontos,Aparições no Top 5,Meses e Posições
0,TEAM LOPES 99,50,1,Agosto (1º)
1,Laranjja Mecannica,40,1,Agosto (2º)
2,FBC Colorado,30,1,Agosto (3º)
3,CosmoCity ZO,20,1,Agosto (4º)
4,HS SPORTS F.C,10,1,Agosto (5º)
